# Ejercicios de Procedimientos y Cursores
**Código de inicialización**

Este código permite inicializar las bibliotecas para usar SQLite en la máquina virtual de Google Colab. El código además elimina cualquier dato almacenado anteriormente en el archvio `ejemplos.db`. Es recomendable volver a correr el código cada vez que se comience con la resolución de un nuevo ejercicio.

In [ ]:
!pip install pymysql
%load_ext sql
!rm -rf ejemplos.db
%sql sqlite:///ejemplos.db

## Ejercicio 16: Consolidación de Feedback de Clientes

**Enunciado:**

Tienes dos tablas: `FeedbackClientes` y `ResumenFeedback`:

```sql
CREATE TABLE FeedbackClientes (
    FeedbackId INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    Comentario VARCHAR(500) NOT NULL,
    Calificacion INT NOT NULL, -- 1 a 5
    FechaFeedback DATE NOT NULL
);

CREATE TABLE ResumenFeedback (
    ResumenId INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    ComentariosTotales INT NOT NULL,
    CalificacionPromedio DECIMAL(3,2) NOT NULL,
    FechaResumen DATE NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `ConsolidarFeedback` que utilice un cursor para procesar los comentarios de los clientes y generar un resumen por cliente.

**Requisitos:**

1. Declarar un cursor que seleccione todos los registros de `FeedbackClientes`.
2. Para cada feedback:
   - Calcular la cantidad total de comentarios y la calificación promedio para cada `ClienteId`.
   - Insertar o actualizar un registro en `ResumenFeedback` con los valores calculados y la fecha actual.
3. Implementar manejo de transacciones para asegurar la consistencia de los datos durante la consolidación.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso.


In [ ]:
# Tablas
%%sql

CREATE TABLE FeedbackClientes (
    FeedbackId INT AUTO_INCREMENT PRIMARY KEY,
    ClienteId INT NOT NULL,
    Comentario VARCHAR(500) NOT NULL,
    Calificacion INT NOT NULL, -- 1 a 5
    FechaFeedback DATE NOT NULL
);

CREATE TABLE ResumenFeedback (
    ResumenId INT AUTO_INCREMENT PRIMARY KEY,
    ClienteId INT NOT NULL,
    ComentariosTotales INT NOT NULL,
    CalificacionPromedio DECIMAL(3,2) NOT NULL,
    FechaResumen DATE NOT NULL
);

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql

DELIMITER //

CREATE PROCEDURE ConsolidarFeedback()
BEGIN
    DECLARE done INT DEFAULT FALSE;
    DECLARE feedbackClienteId INT;
    DECLARE feedbackClienteComentario VARCHAR(255);
    DECLARE feedbackClienteCalificacion INT;
    DECLARE feedbackClienteFecha DATE;
    
    
    DECLARE cursor CURSOR FOR 
    SELECT ClienteId, Comentario, Calificacion, FechaFeedback 
    FROM FeedbackClientes;
    
    DECLARE CONTINUE HANDLER FOR NOT FOUND SET done = TRUE;
    
    START TRANSACTION;

    OPEN cursor;

    read_loop: LOOP
        FETCH cursor INTO feedbackClienteId, feedbackClienteComentario, feedbackClienteCalificacion, feedbackClienteFecha;
        IF done THEN
            LEAVE read_loop;
        END IF;

        INSERT INTO ResumenFeedback (ClienteId, ComentariosTotales, CalificacionPromedio, FechaResumen)
        SELECT 
            feedbackClienteId,
            COUNT(*),
            AVG(Calificacion),
            CURDATE()
        FROM FeedbackClientes
        WHERE ClienteId = feedbackClienteId;   
        END LOOP;

    CLOSE cursor;
    COMMIT;
    
END //